# **Functions and Data**

In [ ]:
#import some python modules
import scipy , itertools, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from google.colab import drive
drive.mount ('/content/drive')

plt.style.use ('bmh'); sns.set()

In [ ]:
#Import data
price = pd.read_csv('/content/drive/MyDrive/01. Materi Kuliah UBP Karawang/Data Mining/Kode/Data/price.csv')

In [ ]:
# Ukuran Data
N, P = price.shape
print('baris = ', N, ', Kolom = ', P)

In [ ]:
# tipe data di setiap kolom
# Wajib di periksa apakah tipe datanya sudah tepat?
# Perhatikan df sebagaimana semua variable di Python diperlakukan seperti object
price.info()

In [ ]:
# "Mengintip" beberapa data pertamanya
price.head(10)

In [ ]:
# "Mengintip" beberapa data akhirnya
price.tail(7)

In [ ]:
# chosen at random
price.sample(8)

# **Removing Variables**

In [ ]:
price.head(10)

In [ ]:
price.describe()

In [ ]:
# Drop kolom pertama karena tidak berguna (hanya index)
price.drop("Observation", axis=1, inplace=True)
price.dtypes

In [ ]:
# Merubah tipe data "jika" tidak tepat, contoh:
price.dtypes

In [ ]:
# dataframe types: https://pbpython.com/pandas_dtypes.html
price['Parking'] = price['Parking'].astype('category')
price['City_Category'] = price['City_Category'].astype('category')
price.dtypes

# **Statiska Deskriptif**

In [ ]:
# Statistika Sederhana dari data "Numerik"-nya
price.describe(include='all')

In [ ]:
# ini adalah parameter tambahan jika kita juga ingin mendapatkan statistik sederhana seluruh datanya
# (termasuk data kategorik)
price[['Dist_Taxi','Parking']].describe(include='all')

# **Variable Selection**

In [ ]:
# Memilih hanya variable dengan tipe tertentu
price_num = price.select_dtypes(include = ['float64', 'int64'])
price_num.head()

In [ ]:
# Memilih hanya variable dengan tipe tertentu
price_cat = price.select_dtypes(include = ['category'])
price_cat.head()

In [ ]:
df = pd.get_dummies(price['Parking'], prefix='Park')
df.head()

# **Menggabungkan dengan Data Awal**

In [ ]:
df2 = pd.concat([price, df], axis = 1)
df2.head(5)

In [ ]:
# Choosing some columns manually
X = price[['House_Price','Dist_Market']] 
X[:10]

# **Distribusi nilai pada setiap variabel kategorik**

In [ ]:
# get all unique values of a variable/column
for col in price_cat.columns:
    print(col,': ', set(price[col].unique()))

In [ ]:
price['Parking'].unique()

In [ ]:
# price
# Jika yang dibutuhkan memang hanya nama kolom, maka kita bisa melakukan hal ini untuk menghemat penggunaan memory
numVar = price.select_dtypes(include = ['float64', 'int64']).columns
list(numVar)

In [ ]:
# Distribusi tiap data
price['Parking'].value_counts()
# kita bisa juga visualisasikan informasi ini

In [ ]:
plt.figure(figsize=(8,6)) # https://matplotlib.org/api/_as_gen/matplotlib.pyplot.figure.html#matplotlib.pyplot.figure
p = sns.countplot(x="Parking", data=price)

In [ ]:
# ini dilakukan jika kita ingin menyimpan plotnya ke dalam suatu file
p.figure.savefig('barChart.png')
# lihat di folder ipynb-nya akan muncul file baru.

In [ ]:
# PieChart
plot = price.Parking.value_counts().plot(kind='pie')

Apakah ada kecenderungan perbedaan harga rumah akibat dari tipe tempat parkir?

In [ ]:
p= sns.catplot(x="Parking", y="House_Price", data=price)
# Apa yang bisa dilihat dari hasil ini?

# **Univariate Outlier removal**

In [ ]:
# Distributions
p = sns.distplot(price['House_Price'], kde=True, rug=True)

In [ ]:
# Misal dengan asumsi data berdistribusi normal
# dan menggunakan 99% confidence interval di sekitar variabel "harga"

df = np.abs(price.House_Price - price.House_Price.mean())<=(2*price.House_Price.std())
# mu-2s<x<mu+2s
print(df.shape)
df.head()

In [ ]:
price2 = price[df] # Data tanpa outliers
print(price2.shape, price.shape)
price2.head
# Perhatikan disini sengaja data yang telah di remove outliernya 
# disimpan dalam variabel baru "Price2"
# Jika datanya besar hati-hati melakukan hal ini

In [ ]:
# Distributions
p = sns.distplot(price2['House_Price'], kde=True, rug=True)

In [ ]:
# Plot lagi setelah outlier removal
p= sns.catplot(x="Parking", y="House_Price", data=price2)
# Apakah ada kecenderungan perbedaan harga rumah akibat dari tipe tempat parkir?

In [ ]:
# Bisa juga plot dengan informasi dari 3 variabel sekaligus
# (untuk melihat kemungkinan faktor interaksi)

p= sns.catplot(x="Parking", y="House_Price", hue="City_Category", kind="swarm", data=price2)

# **Missing Values**

In [ ]:
# General Look at the Missing Values
print(price2.isnull().sum())

In [ ]:
# Simplest solution, if the MV is not a lot
# drop rows with missing values : Ada berbagai cara
X = price.dropna() # jika ada MV minimal satu di salah satu kolom, maka baris di hapus
price2.dropna(how='all') # jika ada MV di semua kolom, maka baris di hapus
price2.dropna(thresh=2) # jika ada MV minimal di salah 2 kolom, maka baris di hapus
price2.dropna(subset=['Dist_Hospital'])[:7] # jika ada MV minimal satu di salah kolom Dist_Hospital
# inplace=True if really really sure
price2.dropna(inplace=True)

In [ ]:
print(price2.isnull().sum())

In [ ]:
#mengecek apakah ada duplikat data?
print(price2.shape)
price2.duplicated().sum()

In [ ]:
#menghapus entri yang memiliki data duplikat 
price2.drop_duplicates(inplace=True)
print(price2.duplicated().sum()) # no more duplicates
print(price2.shape) # re-check by printing data size

# **(PairWise) Scatter Plot**

In [ ]:
# Scatter Plots; https://seaborn.pydata.org/generated/seaborn.pairplot.html
p = sns.pairplot(price2, hue="City_Category")
# Warning agak lambat (variabel cukup banyak)

In [ ]:
# Coba kita perhatikan sebagiannya saja dulu dan coba kelompokkan berdasarkan "Parking"
p = sns.pairplot(price2[['House_Price','Builtup','Dist_Hospital','Dist_Taxi','Parking']], hue="Parking")
# Ada pola menarik?

# **Saving (preprocessed) Data**

In [ ]:
# Saving the preprocessed Data for future use/analysis
price2.to_csv("data_01_pricing_PreProcessed.csv", encoding='utf8')

# **Data Grouping**

In [ ]:
# Slicing DataFrame - Just like query in SQL
price2[price2["City_Category"] == "CAT B"].describe()
# Bisa ditambahkan .drop("Parking", axis=1) untuk menghilangkan kolom dengan single value

# **BoxPlot**

In [ ]:
# BoxPlots
p = sns.boxplot(x="House_Price", y="Parking", data=price2)
# Apa makna pola yang terlihat di data oleh BoxPlot ini?

In [ ]:
# Jika ada outlier grafiknya menjadi tidak jelas (data = price, bukan price2)
p = sns.boxplot(x="House_Price", y="Parking", data=price)

Boxplot dapat juga dipisahkan berdasarkan suatu kategori

In [ ]:
p = sns.catplot(x="Parking", y="House_Price", hue="City_Category", kind="box", data=price2)

# **Checking Correlations**

In [ ]:
price2.corr()

In [ ]:
# HeatMap untuk menyelidiki korelasi
corr2 = price2.corr() # We already examined SalePrice correlations
plt.figure(figsize=(12, 10))

sns.heatmap(corr2[(corr2 >= 0.5) | (corr2 <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
df_iris = pd.read_csv('/content/drive/MyDrive/01. Materi Kuliah UBP Karawang/Data Mining/Kode/Data/iris.csv')
pd.plotting.parallel_coordinates(df_iris, 'species', color=('#556270', '#4ECDC4', '#C7F464'))
plt.show()